# Introduçao

## Prepararando os dados

**Issue relacionada:** [#35](https://github.com/lappis-unb/salic-ml/issues/35)


### Introdução

A **hipótese de que projetos de mesma área e segmentos possuem itens similares** foi negada em relação a métrica definida neste [trabalho](link). Naquele notebook a seguinte métrica foi utilizada para negar esta hipótese:

`p = #(A & B) / #(A | B)`

Onde `A` é o conjunto de itens de um projeto comparado, e `B` é o conjunto de itens do segundo projeto comparado. A hipótese seria verdadeira se e somente se `p >= 0.5`, na média, de um dado subconjunto qualquer da tupla (àrea, segmento).

O valor de `p` é muito sensível a ruídos porque nela cada item do projeto é dado o mesmo peso (por exemplo, peso ou 1). Ou seja, itens muito raros possuem um peso igual a itens muito comuns naquela tupla (area, segmento). Este notebook propõe modificar a métrica acima para confirmar ou negar a hipótese investigada considerando um "peso" para cada tipo de item. Neste passo é assumido como verdadeiro que itens menos frequentes devem possuir um peso menor do que itens mais frequentes.

### Notação


| Notação        | Significado           |
| ------------- |:-------------:|
| `#A` | Número de elementos do conjunto `A` |
| <code>A &#124; B </code>| União dos conjuntos `A` e `B` |
| `A & B` | Intersecção dos conjuntos `A` e `B` |
| `A[i]` | i-ésimo item do conjunto `A`, onde itens são ordenados em ordem crescente |
| `w(i)` | Peso do i-ésimo item do banco de dados. A função `w` atribui os pesos | 

### Hipóteses

**Hipótese 1:** Projetos de mesma área e segmento possuem conjunto de itens similares


**Métricas para confirmar (negar) a hipótese 1**


**Métrica 1**

Chamaremos de `score(s)` a soma dos pesos dos itens (sem repetição) do conjunto `s`. Ou seja, `score(s) = somatorio(w[i])` para todo item `i` em `s`. Em seguida definimos `I = (A & B)` e `U = (A | B)`. Seja `p = (score(I) / score(U))`, a Hipótese 1 será verdadeira se `p >= 0.5`.

**Hipótese 2:** Projetos de segmentos distintos possuem conjunto de itens não-similares

**Métrica 2:**

Se projetos de mesma área e segmentos possuem conjuntos de itens similares (Hipótese 1), mas projetos de segmentos distintos também o possuem, a Hipótese 1 seria pouco útil. Portanto também estamos interessados também na "inversa" da hipótese 1. Considerando as definições da **Métrica 1**, esperamos que a média dos `p` entre projetos de segmentos distintos, seja pele menos **duas vezes menor na média** dos `p`.


**Recarregar automaticamente os módulos**

In [110]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Importando módulos python**

In [111]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats


from salicml.utils.dates import Dates

PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir, os.pardir))
DATA_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')

### Carregando os dados da tabela project_items

In [112]:
items_csv_name = 'project_items.csv'
projects_csv = os.path.join(DATA_FOLDER, items_csv_name)

dt_items = pd.read_csv(projects_csv, low_memory=False)
dt_items.head(1)

,idPronac,PRONAC,Item,idPlanilhaItens,Unidade,QtDias,QtItem,nrOcorrencia,VlUnitarioSolicitado,VlTotalSolicitado,VlUnitarioAprovado,VlTotalAprovado,UfItem,MunicipioItem,Etapa,Area,Segmento,DataProjeto
0,123461,103228,Cartaz,1219,Unidade,180,30.0,6,20.0,3600.0,2.0,360.0,RJ,Rio de Janeiro,3 - Divulgação / Comercialização,3,33,2010-05-06 10:49:13


### Filtrando os dados

É conhecido que dados muito antigos (de 1992, por exemplo) podem ser inconsistentes, então vamos filtrar os dados a partir de uma data de início.

In [113]:
from datetime import datetime

START_DATE = datetime(day=1, month=1, year=2013)

date_column = 'DataProjeto'
dt_items[date_column] = pd.to_datetime(dt_items[date_column], format = Dates.DATE_INPUT_FORMAT)

dt_items = dt_items[dt_items.loc[:, date_column] >= START_DATE]
print(min(dt_items[date_column]))
dt_items.head()

2013-01-02 10:59:07


,idPronac,PRONAC,Item,idPlanilhaItens,Unidade,QtDias,QtItem,nrOcorrencia,VlUnitarioSolicitado,VlTotalSolicitado,VlUnitarioAprovado,VlTotalAprovado,UfItem,MunicipioItem,Etapa,Area,Segmento,DataProjeto
8,202095,160506,Material de consumo,2719,Verba,365,1.0,1,5000.0,5000.0,5000.0,5000.0,MT,Cuiabá,4 - Custos / Administrativos,6,6F,2016-02-25 17:57:15
533,178933,146032,Hospedagem sem alimentação,130,Dia,4,4.0,3,180.0,2160.0,180.0,2160.0,MG,Belo Horizonte,2 - Produção / Execução,1,12,2014-04-11 18:50:56
836,178933,146032,Transporte Local / Locação de Automóvel / Comb...,134,Verba,4,1.0,1,500.0,500.0,500.0,500.0,PB,João Pessoa,2 - Produção / Execução,1,12,2014-04-11 18:50:56
1894,202095,160506,Sítio de Internet - Hospedagem,2601,Verba,365,1.0,1,1068.0,1068.0,1068.0,1068.0,MT,Cuiabá,1 - Pré-Produção / Preparação,6,6F,2016-02-25 17:57:15
2040,202095,160506,Webdesigner,3731,Projeto,1,1.0,1,10000.0,10000.0,10000.0,10000.0,MT,Cuiabá,1 - Pré-Produção / Preparação,6,6F,2016-02-25 17:57:15


## Carregando os dados da tabela projetos

In [114]:
from salicml.models.project_items import Projects

projects = Projects()
projects.dt.head(1)

,IdPRONAC,AnoProjeto,Sequencial,UfProjeto,Area,Segmento,Mecanismo,NomeProjeto,Processo,CgcCpf,...,DtInicioExecucao,DtFimExecucao,SolicitadoUfir,SolicitadoReal,SolicitadoCusteioUfir,SolicitadoCusteioReal,SolicitadoCapitalUfir,SolicitadoCapitalReal,Logon,idProjeto
158164,159372,13,0001,SP,3,33,1,Brasil-Alemanha &#x2013; Quintetos de Sopros,01400000002201381,03521177000121,...,2013-07-01 00:00:00,2013-09-30 00:00:00,0.0,538140.0,0.0,0.0,0.0,0.0,6077.0,93995.0


# Explorando os dados

Explorar rapidamente os dados a fim de obter insights.

**Dados básicos**

In [115]:
dt_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1232336 entries, 8 to 2001716
Data columns (total 18 columns):
idPronac                1232336 non-null int64
PRONAC                  1232336 non-null int64
Item                    1232336 non-null object
idPlanilhaItens         1232336 non-null int64
Unidade                 1232336 non-null object
QtDias                  1232336 non-null int64
QtItem                  1232336 non-null float64
nrOcorrencia            1232336 non-null int64
VlUnitarioSolicitado    1180077 non-null float64
VlTotalSolicitado       1232336 non-null float64
VlUnitarioAprovado      1232336 non-null float64
VlTotalAprovado         1232336 non-null float64
UfItem                  1232336 non-null object
MunicipioItem           1232336 non-null object
Etapa                   1232336 non-null object
Area                    1232336 non-null int64
Segmento                1232336 non-null object
DataProjeto             1232336 non-null datetime64[ns]
dtypes: datetime

# Áreas e projetos mais frequentes

In [116]:
print(min(projects.dt[Projects.DATE].values))
areas = projects.most_frequent_areas()
display(areas[:3])

segments = projects.most_frequent_segments()
display(segments[:3])

most_frequent_area_segment = projects.most_frequent_area_segment()
display(most_frequent_area_segment.head())

2013-01-02T10:59:07.000000000


[(7, 22423), (1, 10112), (3, 8197)]

[('76', 21617), ('11', 7148), ('33', 3485)]

,Area,Segmento,Frequency
118,7,76,21617
0,1,11,7148
49,3,33,3485
47,3,31,3104
59,4,4B,2459


### Verificando se um segmento pode estar em mais de uma área

In [117]:
counts = np.unique(most_frequent_area_segment['Segmento'].values, return_counts=True)
counts = list(zip(*counts))
counts = sorted(counts, key=itemgetter(1), reverse=True)
print(counts[:3])

[('32', 2), ('11', 1), ('12', 1)]


**Um segmento pode estar em mais de uma área**

O Segmento `32` está em 2 áreas.

In [118]:
from salicml.models.project_items import ProjectItems
from salicml.utils.utils import debug

project_items = ProjectItems(dt_items)

In [119]:
ID_PRONAC = 'idPronac'

**Testando a hipótese em cima de uma única tupla**

A tupla de (area, segmento) que possui o maior número de projetos é a tupla `(7, 76)`. Vamos testar, inicialmente, a hipótese em cima deste conjunto de projetos.

In [120]:
AREA, SEGMENTO = ProjectItems.AREA, ProjectItems.SEGMENTO
area_segment = (most_frequent_area_segment.iloc[0].Area, most_frequent_area_segment.iloc[0].Segmento)
print('(area, segmento) = {}'.format(area_segment))

area_and_segment = (dt_items[SEGMENTO] == area_segment[1])
debug('area_and_segment', area_and_segment)
display(dt_items[area_and_segment])

area_and_segment &= (dt_items[AREA] == area_segment[0])
items_area = dt_items.loc[area_and_segment]
display(items_area.head(5))


print('items_area.len = {}'.format(len(items_area.index)))

(area, segmento) = (7, '76')
area_and_segment = 8          False
533        False
836        False
1894       False
2040       False
3012       False
3591       False
4688       False
5419       False
5573       False
5574       False
5604       False
7996       False
7997       False
8052       False
10072      False
10278      False
10326      False
10370      False
10371      False
10430      False
11107      False
11118      False
11119      False
11132      False
11133      False
11212      False
11913      False
12011      False
12219      False
           ...  
2001686    False
2001687    False
2001688    False
2001689    False
2001690    False
2001691    False
2001692    False
2001693    False
2001694    False
2001695    False
2001696    False
2001697    False
2001698    False
2001700    False
2001701    False
2001702    False
2001703    False
2001704    False
2001705    False
2001706    False
2001707    False
2001708    False
2001709    False
2001710    False
2001711    False


,idPronac,PRONAC,Item,idPlanilhaItens,Unidade,QtDias,QtItem,nrOcorrencia,VlUnitarioSolicitado,VlTotalSolicitado,VlUnitarioAprovado,VlTotalAprovado,UfItem,MunicipioItem,Etapa,Area,Segmento,DataProjeto


,idPronac,PRONAC,Item,idPlanilhaItens,Unidade,QtDias,QtItem,nrOcorrencia,VlUnitarioSolicitado,VlTotalSolicitado,VlUnitarioAprovado,VlTotalAprovado,UfItem,MunicipioItem,Etapa,Area,Segmento,DataProjeto


items_area.len = 0


# Importando a tabela Projetos

In [121]:
ID_ITEM = 'idPlanilhaItens'
counts = np.unique(items_area[ID_ITEM].values, return_counts=True)
print('counts[0].size = {}'.format(counts[0].size))
print('max(counts[1]) = {}'.format(max(counts[1])))

counts[0].size = 0


ValueError: max() arg is an empty sequence

**Estatísticas básicas sobre o eixo-y do histograma**

In [ ]:
def plot_bar(y):
    x = np.arange(1, len(y) + 1)
    plt.bar(x, y, align='center')
    plt.xlabel('Id do item')
    plt.ylabel('Número de ocorrências do item')
    plt.title('Histograma de ocorrência de itens em projetos aprovados')
    plt.show()

In [ ]:
from operator import itemgetter

ids_count = list(zip(*counts))
ids_count.sort(key=itemgetter(1), reverse=True)
x, y = zip(*ids_count)
y = np.array(y)
plot_bar(y)

### Sobre o gráfico acima

O eixo x representa uma "normalização" do id dos itens, pois os ids dos itens não necessariamente formam uma permutação de `[1, n]`. Os valores do eixo y foram ordenados em decrescente já que a ordem dos itens não interessa e o objetivo do notebook é investigar os itens mais frequentes. Também é importante comentar que o histograma acima assume que para uma dada planilha orçamentária há apenas uma linha por tipo (id) do item por projeto (idPronac).

### Repetindo o gráfico mas tirando valores de y que estão abaixo do percentile


In [ ]:
PERCENTILE = 50
percentile = np.percentile(y, PERCENTILE)
display(pd.DataFrame(y).describe())
print('percentile = {}'.format(percentile))

y = [val for val in y if val >= percentile]
plot_bar(y)

**Dando peso aos itens**

Neste passo é assumido que itens mais frequentes devem ter um peso maior que itens menos frequentes. Portanto podemos considerar que o peso de um item `x` é o valor `y` no histograma de contagem dos itens. Agora pode-se calcular a **Métrica 1** utilizando estes pesos e enfim confirmar/negar a **Hipótese 1**.

In [ ]:
from collections import Counter

item_weight = Counter(dt_items[ID_ITEM].values)

In [ ]:
def get_items_score(items):
    score = sum(item_weight[id] for id in items)
    return score


def get_project_score(id_pronac):
    items = project_items.items(id_pronac)
    score = get_items_score(items)
    return score


def get_projects_weighted_similarity(itens_a, itens_b):
    intersect = np.intersect1d(itens_a, itens_b)
    union = np.union1d(itens_a, itens_b)
    
    numerator = get_items_score(intersect)
    denominator = get_items_score(union)
    
    similarity = numerator / denominator
    
    assert similarity >= 0.0 and similarity <= 1.0
    return similarity


def get_projects_weighted_similarity_min(itens_a, itens_b):
    if itens_a.size > itens_b.size:
        itens_a, itens_b = itens_b, itens_a
        
    intersect = np.intersect1d(itens_a, itens_b)
    numerator = get_items_score(intersect)
    
    denominator = get_items_score(itens_a)
    
    similarity = numerator / denominator
    
    assert similarity >= 0.0 and similarity <= 1.0
    return similarity

**Calculando a similaridade entre projetos de áreas (1, '6F')**

Devido o custo computacional das operações na tabela/dataframe, é inviável calcular a similaridade entre todas as combinações tomadas 2 a 2 de todos os projetos da área acima. Um contorno possível é selecionar um subconjunto aleatório destes projetos e testar todas as combinações tomadas 2 a 2 deste subconjunto.

In [ ]:
PERCENT =  0.025
pronacs_in_area = items_area[ID_PRONAC].unique()

%store -r pronacs_sample

pronacs = pronacs_sample
print('sample size = {}'.format(pronacs.size))

In [ ]:
%%time
size = pronacs.size
sims = []
sims_min = []

for i in range(size):
    id1 = pronacs[i]
    items_a = project_items.items(id1, dt=items_area)

    for j in range(i + 1, size):
        id2 = pronacs[j]
        items_b = project_items.items(id2, items_area)
        similarity = get_projects_weighted_similarity(items_a, items_b)
        similarity_min = get_projects_weighted_similarity_min(items_a, items_b)
        
        sims.append(similarity)
        sims_min.append(similarity_min)

similarities = np.array(sims, float)
similarities_min = np.array(sims_min, float)


**Resultados da similaridade em relação ao tamanho da união dos conjuntos**

In [ ]:
weighed_sim_stats = pd.DataFrame(similarities).describe()
display(weighed_sim_stats)

In [ ]:
plt.hist(similarities)
plt.title('Similaridade considerando o conjunto união dos itens')
plt.xlabel('Similaridade em [0, 1]')
plt.ylabel('Número de combinações, C(n, 2)')
plt.show()

In [ ]:
plt.hist(similarities_min)
plt.title('Similaridade considerando o menor conjunto de itens da combinação')
plt.xlabel('Similaridade em [0, 1]')
plt.ylabel('Número de combinações, C(n, 2)')
plt.show()

# Comparação com os resultados da mesma métrica mas sem pesos

**Métrica 1**

In [ ]:
%store -r sim_describe


dt_comp = pd.DataFrame(sim_describe)
dt_comp['comparação ponderada'] = weighed_sim_stats[0].values
dt_comp.rename(columns = {0: 'comparação sem ponderar'}, inplace=True)
display(dt_comp)

### Conclusões

Na média, a similaridade dos projetos aumentou, de `0.154977` para `0.285552`. É preciso ainda testar a Hipótese 2 para se comparar estes valores a fim de fato negar ou confirmar a Hipótese 1, pois pode ser que a similaridade entre projetos de segmentos distintos seja muito menor do que a similaridade entre projetos de mesmos segmentos.

#### Contribuições futuras

- Negar ou confirmar a Hipótese 2
- Considerar uma métrica similar ao `tf-idf`, descrita em https://en.wikipedia.org/wiki/Tf%E2%80%93idf